In [25]:
from __future__ import print_function, division
import pandas as pd 
import numpy as np
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
dtypes = {
    'id': np.int32,
    'year': np.int32,
    'month': np.int8,
    'id': 'object',
    'url': 'object',
    'word_count': np.float16,
    'article': 'object'
    ''
}
raw_df = pd.read_csv('./data/11_17_final.csv', names=['ind', 'year', 'month', 'id', 'url', 'word_count', 'article'], dtype=dtypes, index_col=False)

In [3]:
raw_df.drop_duplicates(subset='id', inplace=True)
raw_df = raw_df.dropna().reset_index()

In [4]:
subset_size = 20000
subset_df = raw_df.groupby('year').apply(lambda x: x.sample(subset_size)).reset_index(drop=True)

In [5]:
article_text = subset_df['article']

In [6]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(article_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
# Create the term-document matrix
# Transpose it so the terms are the rows
counts = count_vectorizer.transform(article_text).transpose()

In [8]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(counts)

In [ ]:
# We need to save a mapping (dict) of row id to word (token) for later use by gensim:

id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [ ]:
# Create lda model (equivalent to "fit" in sklearn)
NUM_TOPICS= 18
lda = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=id2word, passes=10, eval_every=None)
# lda = models.LdaMulticore(corpus=corpus, num_topics=15, id2word=id2word, passes=10)

# IMPORTANT NOTE: There's an eval_every parameter that should control how often gensim runs a perplexity evaluation,
# which is a time expensive step. It seems to me that it isn't working properly, and the only way to really
# get a speed up is to set it to None.

2018-03-08 05:14:13,260 : INFO : using symmetric alpha at 0.05555555555555555
2018-03-08 05:14:13,261 : INFO : using symmetric eta at 0.05555555555555555
2018-03-08 05:14:16,344 : INFO : using serial LDA version on this node
2018-03-08 05:15:13,752 : INFO : running online (multi-pass) LDA training, 18 topics, 10 passes over the supplied corpus of 140000 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 50x with a convergence threshold of 0.001000
2018-03-08 05:15:13,952 : INFO : PROGRESS: pass 0, at document #2000/140000
2018-03-08 05:15:25,874 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 05:16:20,104 : INFO : topic #1 (0.056): 0.008*"said" + 0.004*"mr" + 0.002*"year" + 0.002*"like" + 0.001*"people" + 0.001*"new" + 0.001*"years" + 0.001*"ms" + 0.001*"time" + 0.001*"just"
2018-03-08 05:16:20,333 : INFO : topic #15 (0.056): 0.007*"said" + 0.006*"mr" + 0.002*"new" + 0.002*"year" + 0.002*"like

2018-03-08 05:22:16,600 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 05:23:11,122 : INFO : topic #11 (0.056): 0.006*"said" + 0.002*"people" + 0.002*"mr" + 0.002*"like" + 0.002*"new" + 0.001*"ms" + 0.001*"just" + 0.001*"years" + 0.001*"time" + 0.001*"did"
2018-03-08 05:23:11,445 : INFO : topic #0 (0.056): 0.010*"said" + 0.003*"people" + 0.003*"police" + 0.003*"government" + 0.002*"killed" + 0.002*"forces" + 0.002*"military" + 0.002*"security" + 0.002*"mr" + 0.002*"al"
2018-03-08 05:23:11,684 : INFO : topic #9 (0.056): 0.005*"said" + 0.004*"new" + 0.003*"mr" + 0.003*"like" + 0.002*"company" + 0.001*"google" + 0.001*"year" + 0.001*"york" + 0.001*"new york" + 0.001*"people"
2018-03-08 05:23:11,984 : INFO : topic #10 (0.056): 0.006*"said" + 0.006*"game" + 0.004*"season" + 0.003*"points" + 0.003*"team" + 0.003*"games" + 0.002*"second" + 0.002*"play" + 0.002*"win" + 0.002*"year"
2018-03-08 05:23:12,279 : INFO : topic #3 (0.056): 0.010*"said" + 0.006

2018-03-08 05:30:03,661 : INFO : topic #10 (0.056): 0.007*"game" + 0.007*"said" + 0.004*"season" + 0.004*"team" + 0.004*"points" + 0.003*"games" + 0.003*"play" + 0.003*"coach" + 0.003*"second" + 0.002*"players"
2018-03-08 05:30:03,975 : INFO : topic #15 (0.056): 0.014*"mr" + 0.011*"said" + 0.003*"news" + 0.002*"case" + 0.002*"police" + 0.002*"court" + 0.002*"new" + 0.002*"law" + 0.002*"company" + 0.002*"did"
2018-03-08 05:30:04,320 : INFO : topic #12 (0.056): 0.008*"mr" + 0.004*"new" + 0.004*"said" + 0.003*"music" + 0.003*"like" + 0.002*"film" + 0.002*"school" + 0.002*"theater" + 0.002*"ms" + 0.002*"york"
2018-03-08 05:30:05,122 : INFO : topic diff=0.376388, rho=0.277350
2018-03-08 05:30:05,390 : INFO : PROGRESS: pass 0, at document #28000/140000
2018-03-08 05:30:17,155 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 05:31:11,645 : INFO : topic #1 (0.056): 0.011*"said" + 0.007*"ms" + 0.004*"like" + 0.003*"people" + 0.003*"just" + 0.003*"mr" + 0.

2018-03-08 05:36:52,576 : INFO : topic #4 (0.056): 0.013*"percent" + 0.008*"said" + 0.007*"year" + 0.004*"company" + 0.003*"million" + 0.003*"market" + 0.003*"companies" + 0.003*"billion" + 0.002*"sales" + 0.002*"new"
2018-03-08 05:36:53,393 : INFO : topic diff=0.380798, rho=0.229416
2018-03-08 05:36:53,681 : INFO : PROGRESS: pass 0, at document #40000/140000
2018-03-08 05:37:05,247 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 05:37:59,973 : INFO : topic #12 (0.056): 0.008*"mr" + 0.004*"new" + 0.003*"film" + 0.003*"music" + 0.003*"said" + 0.003*"like" + 0.002*"art" + 0.002*"theater" + 0.002*"movie" + 0.002*"ms"
2018-03-08 05:38:00,211 : INFO : topic #9 (0.056): 0.004*"like" + 0.004*"said" + 0.004*"company" + 0.003*"new" + 0.002*"mr" + 0.002*"online" + 0.002*"web" + 0.002*"apple" + 0.002*"use" + 0.002*"facebook"
2018-03-08 05:38:00,432 : INFO : topic #14 (0.056): 0.006*"said" + 0.005*"european" + 0.004*"bank" + 0.004*"china" + 0.003*"governmen

2018-03-08 05:43:52,304 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 05:44:46,836 : INFO : topic #16 (0.056): 0.027*"mr" + 0.008*"said" + 0.006*"president" + 0.005*"obama" + 0.004*"political" + 0.004*"party" + 0.003*"campaign" + 0.003*"mr obama" + 0.002*"israel" + 0.002*"election"
2018-03-08 05:44:47,223 : INFO : topic #12 (0.056): 0.009*"mr" + 0.003*"new" + 0.003*"music" + 0.003*"film" + 0.003*"like" + 0.002*"said" + 0.002*"art" + 0.002*"theater" + 0.002*"movie" + 0.002*"ms"
2018-03-08 05:44:47,633 : INFO : topic #7 (0.056): 0.006*"mr" + 0.004*"north" + 0.003*"south" + 0.003*"korea" + 0.002*"said" + 0.002*"korean" + 0.002*"north korea" + 0.001*"kim" + 0.001*"south korea" + 0.001*"party"
2018-03-08 05:44:47,885 : INFO : topic #1 (0.056): 0.012*"said" + 0.007*"ms" + 0.005*"like" + 0.004*"people" + 0.003*"just" + 0.003*"time" + 0.003*"don" + 0.003*"life" + 0.003*"years" + 0.003*"school"
2018-03-08 05:44:48,132 : INFO : topic #14 (0.056): 0.005*

...

2018-03-08 16:06:57,473 : INFO : topic #3 (0.056): 0.009*"said" + 0.007*"state" + 0.004*"new" + 0.003*"mr" + 0.003*"law" + 0.003*"federal" + 0.002*"tax" + 0.002*"house" + 0.002*"people" + 0.002*"republican"
2018-03-08 16:06:58,276 : INFO : topic diff=0.244675, rho=0.112509
2018-03-08 16:06:58,530 : INFO : PROGRESS: pass 8, at document #46000/140000
2018-03-08 16:07:08,325 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 16:08:02,868 : INFO : topic #9 (0.056): 0.005*"company" + 0.005*"like" + 0.003*"new" + 0.003*"said" + 0.003*"online" + 0.002*"web" + 0.002*"people" + 0.002*"use" + 0.002*"site" + 0.002*"internet"
2018-03-08 16:08:03,217 : INFO : topic #17 (0.056): 0.006*"city" + 0.006*"said" + 0.005*"new" + 0.005*"mr" + 0.004*"street" + 0.003*"building" + 0.003*"house" + 0.002*"park" + 0.002*"york" + 0.002*"new york"
2018-03-08 16:08:03,610 : INFO : topic #10 (0.056): 0.007*"game" + 0.006*"said" + 0.005*"season" + 0.004*"team" + 0.004*"points" + 0

2018-03-08 16:14:44,184 : INFO : topic #2 (0.056): 0.010*"said" + 0.004*"united" + 0.004*"states" + 0.003*"united states" + 0.003*"officials" + 0.003*"american" + 0.002*"nuclear" + 0.002*"agency" + 0.002*"power" + 0.002*"energy"
2018-03-08 16:14:44,603 : INFO : topic #7 (0.056): 0.004*"food" + 0.003*"north" + 0.003*"south" + 0.002*"korea" + 0.002*"restaurant" + 0.002*"korean" + 0.002*"wine" + 0.001*"kim" + 0.001*"meat" + 0.001*"chef"
2018-03-08 16:14:44,982 : INFO : topic #4 (0.056): 0.013*"percent" + 0.009*"said" + 0.007*"year" + 0.005*"company" + 0.005*"million" + 0.004*"market" + 0.003*"billion" + 0.003*"companies" + 0.003*"sales" + 0.002*"years"
2018-03-08 16:14:45,466 : INFO : topic #13 (0.056): 0.007*"new" + 0.006*"mr" + 0.004*"said" + 0.004*"art" + 0.003*"york" + 0.003*"new york" + 0.003*"museum" + 0.003*"million" + 0.002*"director" + 0.002*"group"
2018-03-08 16:14:45,798 : INFO : topic #1 (0.056): 0.016*"said" + 0.007*"like" + 0.006*"ms" + 0.005*"just" + 0.005*"people" + 0.004*

2018-03-08 16:21:27,025 : INFO : topic #14 (0.056): 0.006*"china" + 0.005*"european" + 0.005*"bank" + 0.004*"government" + 0.004*"chinese" + 0.004*"said" + 0.003*"europe" + 0.003*"financial" + 0.003*"countries" + 0.002*"union"
2018-03-08 16:21:27,446 : INFO : topic #16 (0.056): 0.030*"mr" + 0.007*"president" + 0.006*"said" + 0.005*"obama" + 0.005*"party" + 0.004*"political" + 0.003*"campaign" + 0.003*"mr obama" + 0.003*"russia" + 0.002*"election"
2018-03-08 16:21:27,714 : INFO : topic #6 (0.056): 0.005*"said" + 0.005*"game" + 0.004*"season" + 0.004*"run" + 0.003*"yankees" + 0.003*"baseball" + 0.003*"hit" + 0.003*"runs" + 0.003*"league" + 0.002*"team"
2018-03-08 16:21:28,520 : INFO : topic diff=0.251377, rho=0.112509
2018-03-08 16:21:28,770 : INFO : PROGRESS: pass 8, at document #72000/140000
2018-03-08 16:21:38,612 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 16:22:33,194 : INFO : topic #9 (0.056): 0.005*"company" + 0.005*"like" + 0.003*"new"

2018-03-08 16:28:10,016 : INFO : topic #8 (0.056): 0.007*"dr" + 0.007*"university" + 0.004*"students" + 0.004*"new" + 0.003*"health" + 0.003*"school" + 0.003*"medical" + 0.003*"college" + 0.002*"study" + 0.002*"york"
2018-03-08 16:28:10,816 : INFO : topic diff=0.258797, rho=0.112509
2018-03-08 16:28:11,085 : INFO : PROGRESS: pass 8, at document #84000/140000
2018-03-08 16:28:20,824 : INFO : merging changes from 2000 documents into a model of 140000 documents
2018-03-08 16:29:15,485 : INFO : topic #15 (0.056): 0.020*"mr" + 0.014*"said" + 0.005*"police" + 0.004*"case" + 0.004*"court" + 0.002*"department" + 0.002*"judge" + 0.002*"did" + 0.002*"ms" + 0.002*"law"
2018-03-08 16:29:15,828 : INFO : topic #12 (0.056): 0.009*"mr" + 0.004*"music" + 0.003*"film" + 0.003*"new" + 0.002*"theater" + 0.002*"movie" + 0.002*"like" + 0.001*"ms" + 0.001*"dance" + 0.001*"festival"
2018-03-08 16:29:16,244 : INFO : topic #2 (0.056): 0.009*"said" + 0.004*"states" + 0.004*"united" + 0.004*"united states" + 0.00

2018-03-08 16:35:58,677 : INFO : topic #10 (0.056): 0.007*"game" + 0.005*"season" + 0.005*"said" + 0.004*"team" + 0.004*"points" + 0.003*"play" + 0.003*"games" + 0.003*"coach" + 0.002*"second" + 0.002*"players"
2018-03-08 16:35:58,983 : INFO : topic #6 (0.056): 0.005*"game" + 0.004*"said" + 0.004*"season" + 0.004*"run" + 0.003*"mets" + 0.003*"yankees" + 0.003*"hit" + 0.003*"baseball" + 0.002*"runs" + 0.002*"league"
2018-03-08 16:35:59,457 : INFO : topic #4 (0.056): 0.013*"percent" + 0.008*"said" + 0.007*"year" + 0.005*"company" + 0.005*"million" + 0.004*"market" + 0.003*"billion" + 0.003*"companies" + 0.002*"years" + 0.002*"new"
2018-03-08 16:35:59,915 : INFO : topic #3 (0.056): 0.009*"said" + 0.007*"state" + 0.004*"new" + 0.003*"law" + 0.003*"mr" + 0.003*"federal" + 0.002*"people" + 0.002*"city" + 0.002*"republican" + 0.002*"house"
2018-03-08 16:36:00,458 : INFO : topic #12 (0.056): 0.009*"mr" + 0.003*"music" + 0.003*"film" + 0.003*"new" + 0.002*"theater" + 0.002*"movie" + 0.002*"like

In [31]:
# Here are the 20 most important words for each of the 5 topics we found:
all_topics = np.array(lda.print_topics(num_words=15))

2018-03-08 19:42:46,926 : INFO : topic #0 (0.056): 0.010*"said" + 0.004*"government" + 0.004*"people" + 0.003*"military" + 0.003*"united" + 0.003*"state" + 0.003*"country" + 0.002*"officials" + 0.002*"security" + 0.002*"killed" + 0.002*"american" + 0.002*"police" + 0.002*"war" + 0.002*"attack" + 0.002*"islamic"
2018-03-08 19:42:47,396 : INFO : topic #1 (0.056): 0.014*"said" + 0.007*"like" + 0.006*"ms" + 0.005*"people" + 0.005*"just" + 0.004*"time" + 0.004*"don" + 0.003*"know" + 0.003*"years" + 0.003*"think" + 0.003*"way" + 0.003*"life" + 0.003*"women" + 0.003*"day" + 0.003*"going"
2018-03-08 19:42:47,794 : INFO : topic #2 (0.056): 0.009*"said" + 0.005*"united" + 0.005*"states" + 0.004*"united states" + 0.003*"north" + 0.003*"american" + 0.002*"water" + 0.002*"officials" + 0.002*"climate" + 0.002*"nuclear" + 0.002*"power" + 0.002*"agency" + 0.002*"energy" + 0.002*"air" + 0.002*"administration"
2018-03-08 19:42:48,249 : INFO : topic #3 (0.056): 0.009*"said" + 0.005*"state" + 0.004*"new" 

In [32]:
for topic in all_topics:
    print('topic #{}'.format(topic[0]))
    for word_weight in topic[1].split('+'):
        [weight, word] = word_weight.split('*')
        print('{}: {}'.format(word, weight))

topic #0
"said" : 0.010
"government" :  0.004
"people" :  0.004
"military" :  0.003
"united" :  0.003
"state" :  0.003
"country" :  0.003
"officials" :  0.002
"security" :  0.002
"killed" :  0.002
"american" :  0.002
"police" :  0.002
"war" :  0.002
"attack" :  0.002
"islamic":  0.002
topic #1
"said" : 0.014
"like" :  0.007
"ms" :  0.006
"people" :  0.005
"just" :  0.005
"time" :  0.004
"don" :  0.004
"know" :  0.003
"years" :  0.003
"think" :  0.003
"way" :  0.003
"life" :  0.003
"women" :  0.003
"day" :  0.003
"going":  0.003
topic #2
"said" : 0.009
"united" :  0.005
"states" :  0.005
"united states" :  0.004
"north" :  0.003
"american" :  0.003
"water" :  0.002
"officials" :  0.002
"climate" :  0.002
"nuclear" :  0.002
"power" :  0.002
"agency" :  0.002
"energy" :  0.002
"air" :  0.002
"administration":  0.002
topic #3
"said" : 0.009
"state" :  0.005
"new" :  0.004
"law" :  0.003
"people" :  0.003
"federal" :  0.003
"mr" :  0.003
"tax" :  0.003
"house" :  0.003
"health" :  0.003
"ca

In [34]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [ ]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [ ]:
# Check out some of the document vectors in the topic space
# for i, k in enumerate(lda_docs[0:20]):
#     print(i,k)

In [ ]:
# lda_docs[0][1]
# lda_corpus[0]

In [35]:
def get_corpus_for_year(yr):
    articles_yr = raw_df[raw_df['year'] == yr]['article']
    print('number of articles for year {}: {}'.format(yr, len(articles_yr)))
    counts_yr = count_vectorizer.transform(articles_yr).transpose()
    corpus_yr = matutils.Sparse2Corpus(counts_yr)
    lda_corpus_yr = lda[corpus_yr]
#     lda_docs_yr = [doc for doc in lda_corpus_yr]
    print('done with year {}'.format(yr))
    return lda_corpus_yr

def corpus_to_dic(lda_corpus_yr):
    lda_docs_yr = [{tpl[0]: tpl[1] for tpl in doc} for doc in lda_corpus_yr]
    return lda_docs_yr

In [36]:
def get_avg_weight_for_year(year_corpus, num_topics):
    list_of_sums = np.zeros(num_topics)
    for lda_doc in year_corpus:
        for weight in lda_doc:
            list_of_sums[weight[0]] += weight[1]
    average_weights = list_of_sums/len(year_corpus)
    return average_weights

In [ ]:
corpus_2011 = get_corpus_for_year(2011)
corpus_2012 = get_corpus_for_year(2012)
corpus_2013 = get_corpus_for_year(2013)
corpus_2014 = get_corpus_for_year(2014)
corpus_2015 = get_corpus_for_year(2015)
corpus_2016 = get_corpus_for_year(2016)
corpus_2017 = get_corpus_for_year(2017)

number of articles for year 2011: 36103
done with year 2011
number of articles for year 2012: 36057
done with year 2012
number of articles for year 2013: 34795
done with year 2013
number of articles for year 2014: 40790


In [ ]:
dicts_2011 = corpus_to_dic(corpus_2011)
dicts_2012 = corpus_to_dic(corpus_2012)
dicts_2013 = corpus_to_dic(corpus_2013)
dicts_2014 = corpus_to_dic(corpus_2014)
dicts_2015 = corpus_to_dic(corpus_2015)
dicts_2016 = corpus_to_dic(corpus_2016)
dicts_2017 = corpus_to_dic(corpus_2017)

In [ ]:
avg_weights_2011 = get_avg_weight_for_year(corpus_2011, NUM_TOPICS)
avg_weights_2012 = get_avg_weight_for_year(corpus_2012, NUM_TOPICS)
avg_weights_2013 = get_avg_weight_for_year(corpus_2013, NUM_TOPICS)
avg_weights_2014 = get_avg_weight_for_year(corpus_2014, NUM_TOPICS)
avg_weights_2015 = get_avg_weight_for_year(corpus_2015, NUM_TOPICS)
avg_weights_2016 = get_avg_weight_for_year(corpus_2016, NUM_TOPICS)
avg_weights_2017 = get_avg_weight_for_year(corpus_2017, NUM_TOPICS)

In [ ]:
def get_weight_for_doc(doc, topic_id):
    try:
        return doc[topic_id]
    except:
        return 0
    
def get_top_articles_index(year_dict, topic_id ):
    return sorted(enumerate(year_dict), key=lambda x: get_weight_for_doc(x[1], topic_id), reverse=True)

def get_article(year, idx):
    return raw_df[raw_df['year'] == year]['article'].reset_index(drop=True)[idx]

def get_articles_per_topic(year_dict, topic_id, year, lim=5):
    rows = get_top_articles_index(year_dict, topic_id )
#     print(rows[1:10])
    for idx, row in enumerate(rows[0:lim]):
        print('Article {}'.format(idx + 1))
        print('Score: {}'.format(row[1][topic_id]))
        print(get_article(year, row[0])[0:500])

In [52]:
for i in range(0,NUM_TOPICS):
    print('\n\nTOPIC {}'.format(i))
    get_articles_per_topic(dicts_2012, i, 2012, lim=11)




TOPIC 0
Article 1
Score: 0.9352463483810425
ADEN, Yemen (Reuters) — The Yemeni Army battled militants linked to Al Qaeda in the southern town of Zinjibar on Saturday, recapturing important positions in the rebel-held city and killing at least 62 Islamist fighters, a military official said. The official said four government soldiers died and four were wounded in the fighting, part of an offensive that began this month to uproot Islamist militants from southern Yemen. He said many of the dead militants were Somalis. The militant group Ansar
Article 2
Score: 0.9317195415496826
SANA, Yemen (AP) — The death toll from a suicide bombing in southern Yemen rose to 45 on Sunday, officials said, in the latest attack against militias allied with the army. The bomber, suspected of being a member of Al Qaeda, struck late Saturday during a funeral service attended by members of civilian militias that helped the Yemeni Army in a campaign to recapture the town of Jaar from Qaeda militants in June. Th

RECENTLY, as I heard my daughter, a junior in high school, and her friends discuss their plans for the prom, I had a vaguely troubling thought: can a 16-year-old be a cougar? Her best friends wanted to take boys younger than themselves (much younger ... two entire grades younger) to the prom. And one of those boys just happened to be my ninth-grade son. Back in my prom days (when the big slow dance was still “Stairway to Heaven”), I went with a boy who was not just taller than me, but older as w
Article 10
Score: 0.7878271341323853
“WILL he have a normal family?” Andrea asked. After the demise of her long-term relationship, my friend had enlisted a psychic in a search for answers. Never having felt close to her ex’s parents, she was hoping to avoid a repeat with whoever came next. “No one has a normal family,” the psychic said. “Normal is not a word.” Andrea started to say something else, but he cut her off: “Get in the back seat, honey. You’re busting up my flow.” We were seated in a 

Article 1
Score: 0.9639773368835449
Honeywell International said on Friday that its profit rose 10 percent in the third quarter as new products and emerging market growth offset weakness in Europe. The earnings topped Wall Street estimates, but revenue fell short of estimates. The company reduced its full-year revenue guidance and narrowed its earnings estimate. Honeywell’s chief executive, David M. Cote, said in a statement that he was encouraged by the commercial aerospace outlook, increasing spending on infrastructure and oil a
Article 2
Score: 0.9537493586540222
Tiffany & Company reported Thursday that its third-quarter net income fell about 30 percent, citing a higher-than-expected tax rate, economic weakness and high precious metal and diamond costs. The jewelry company’s results missed Wall Street’s expectations, and it cut its full-year earnings forecast. Its shares dropped 6.2 percent. For the quarter ended Oct. 31, Tiffany earned $63.2 million, or 49 cents a share, compared w

Article 1
Score: 0.9318841099739075
PHOENIX — Ike Davis had four hits and became the ninth player in Mets history to hit three home runs in a game. But even that was not enough for the slumping Mets, who lost to the Arizona Diamondbacks, 6-3, on Saturday at Chase Field. “A huge night for him,” Manager Terry Collins said of Davis. “We didn’t give him any help.” Davis led off the second inning and crushed a ball to straightaway center field. In the fourth, he pounded another to roughly the same spot. Davis then led off the sixth and
Article 2
Score: 0.9146016836166382
Josh Beckett had five strikeouts and one confrontation with an umpire in six sharp innings and David Ortiz homered and had three runs batted in to lead the Boston Red Sox to an 11-2 romp over the host Minnesota Twins on Tuesday night. Beckett (2-2) allowed two runs and five hits and Mike Aviles went 4 for 5 with a home run and two R.B.I. for the Red Sox, who had no trouble with this big lead after squandering a nine-run adv

Article 1
Score: 0.9715853333473206
Dr. Marisa Rory Mendel, the daughter of Ilene S. Mendel and Dr. Herbert E. Mendel of Fayetteville, N.Y., was married Saturday evening in Syracuse to Joshua Michael Abraham, a son of Harriet Sostman Abraham and Dr. Jerrold L. Abraham, also of Fayetteville. Rabbi Charles S. Sherman performed the ceremony at Temple Adath Yeshurun. The bride, 28, will continue to use her name professionally. She is a psychiatry resident at Brigham and Women’s Hospital and at Beth Israel Deaconess Medical Center, bot
Article 2
Score: 0.9607828259468079
Dr. Alisse Katherine Hauspurg and Dr. Adam James Janicki were married Saturday at Saltwater Farm Vineyard in Stonington, Conn. The ceremony was led by Jim R. Moss, an uncle of the bridegroom, who became a Universal Life Minister for the event. The bride, 26, and the bridegroom, 25, met at the University of Pennsylvania, from which each graduated magna cum laude. The bride will continue to use her name professionally. Later 

Article 1
Score: 0.9846483469009399
Dirk Nowitzki scored 16 points, driving past Kevin Garnett for the go-ahead layup with 5.1 seconds left and leading the Dallas Mavericks to a 90-85 victory over the host Boston Celtics on Wednesday night. Dallas led most of the game, but Paul Pierce hit a 3-pointer — just his second basket of the night — to tie it at 85-85 with 25 seconds to play. Rajon Rondo scored 24 points for Boston, and Garnett had 16 points with 10 rebounds. NUGGETS 123, NETS 115 Danilo Gallinari scored 22 points and Arron
Article 2
Score: 0.9826453924179077
Danilo Gallinari scored a season-high 28 points and outplayed Tim Duncan down the stretch as the host Denver Nuggets beat the San Antonio Spurs, 112-106, on Tuesday night. Duncan had 31 points, also a season high, and 18 rebounds for the Spurs, who cut a 15-point lead to 3 with 3 minutes 38 seconds left before the Nuggets closed it out for their third straight win. Manu Ginobili scored 16 points for the Spurs, who have lost

Article 1
Score: 0.8950509428977966
‘DARKNESSE VISIBLE’ Inon Barnatan, pianist. Avie AV2256; CD. EACH of the pieces on “Darknesse Visible,” a new recording by the brilliant pianist Inon Barnatan, was inspired by a literary work. The thoughtful program is typical for this insightful musician. But Mr. Barnatan’s extraordinary playing is what makes the release so rewarding. The three pieces of Ravel’s “Gaspard de la Nuit” (which roughly translates as “Treasurer of the Night”) take their titles from poems by Aloysius Bertrand. In this
Article 2
Score: 0.8950509428977966
‘DARKNESSE VISIBLE’ Inon Barnatan, pianist. Avie AV2256; CD. EACH of the pieces on “Darknesse Visible,” a new recording by the brilliant pianist Inon Barnatan, was inspired by a literary work. The thoughtful program is typical for this insightful musician. But Mr. Barnatan’s extraordinary playing is what makes the release so rewarding. The three pieces of Ravel’s “Gaspard de la Nuit” (which roughly translates as “Treasurer 

Tim Allen joined the Boston office of SapientNitro, part of Sapient, in a new post, creative director for experience design. He had been a creative director at R/GA, New York, part of the Interpublic Group of Companies. Association of Hispanic Advertising Agencies, McLean, Va., was renamed AHAA: The Voice of Hispanic Marketing as it expands its membership to include agencies that specialize in tasks aimed at Spanish-speaking consumers like public relations, direct response and promotions as well
Article 7
Score: 0.8255565762519836
Kate Bittman joined the New York office of Rodale as associate director for communications at Men’s Health magazine. She had been public relations manager at The New Yorker magazine, part of the Condé Nast Publications division of Advance Publications. James Brooks joined Forge Worldwide, Boston, as an interactive art director. He had been a senior designer at the Boston office of Digitas, part of the VivaKi division of the Publicis Groupe. Andrew Campbell an

TOPIC 15
Article 1
Score: 0.89580899477005
A New York City police officer accused of providing protected law enforcement information to the head of a drug-dealing organization in Jamaica, Queens, was indicted on Friday by a grand jury in Federal District Court in Brooklyn. Police Officer Devon Daniels was charged with four counts of accessing a computer database without authorization and four counts of making false statements to federal agents during their investigation. He faces up to 10 years in prison on each of the computer charges a
Article 2
Score: 0.8695391416549683
A former New York City police officer was sentenced on Thursday to more than three years in prison in a gun smuggling case that ensnared active-duty and other former members of the Police Department. The former officer, Joseph Trischitta, 43, of Staten Island, was sentenced in Federal District Court in Manhattan to 40 months in prison. He pleaded guilty in February to conspiracy to transport firearms between states, 

Article 1
Score: 0.9498709440231323
EMIGRATION CANYON, UTAH WHAT: A two-bedroom, one-bath house HOW MUCH: $342,500 SIZE: 1,008 square feet PRICE PER SQUARE FOOT: $339.78 SETTING: Emigration Canyon is an unincorporated community of 1,600 people nestled in the Wasatch Mountains, about eight miles from downtown Salt Lake City. This airy remodeled house is one of several along Emigration Canyon Road, the main route winding through the canyon. Nearby parks and nature areas include Red Butte Canyon and Wasatch Mountain State Park. INSID
Article 2
Score: 0.9458158016204834
MINNEAPOLIS WHAT: A five-bedroom house with four baths HOW MUCH: $1,995,000 SIZE: 5,008 square feet PRICE PER SQUARE FOOT: $398.36 SETTING: This house is in a neighborhood called Kenwood, set between Cedar Lake and Lake of the Isles, two lakes in a section of Minneapolis called the Chain of Lakes. Hockey and ice-skating are popular during the winter; canoeing, swimming and fishing take over during the summer. Kenwood is kno

In [60]:
lda_topics = ['Middle East Violence', 'Op-Ed (personal stories)', 'Energy', 'Telephone polling results', 'Finance', 'Bridge Card Game', 'Baseball', 'Culinary', 'Marriages', 'Technology', 'Basketball', 'Random', 'Art', 'Business Acquisitions/Hires', 'International Economics', 'Crime', 'International Politics', 'Real Estate Sales']

In [58]:
topic_words = []
for i in range(0,17):
    topic_words.append(lda.print_topic(i, topn=20))

In [77]:
for topic_list in zip(lda_topics, topic_words):
    print(topic_list[0])
    for word in topic_list[1].split('+'):
        word_weight = word.strip().split('*')
        print('{}\t{}'. format(int(1000* float(word_weight[0])), word_weight[1].strip('"')))
    print('\n')

    

Middle East Violence
10	said
4	government
4	people
3	military
3	united
3	state
3	country
2	officials
2	security
2	killed
2	american
2	police
2	war
2	attack
2	islamic
2	group
2	forces
2	islamic state
2	syria
2	attacks


Op-Ed (personal stories)
14	said
7	like
6	ms
5	people
5	just
4	time
4	don
3	know
3	years
3	think
3	way
3	life
3	women
3	day
3	going
3	really
3	want
3	did
2	work
2	ve


Energy
9	said
5	united
5	states
4	united states
3	north
3	american
2	water
2	officials
2	climate
2	nuclear
2	power
2	agency
2	energy
2	air
2	administration
2	change
1	oil
1	korea
1	safety
1	gas


Telephone polling results
9	said
5	state
4	new
3	law
3	people
3	federal
3	mr
3	tax
3	house
3	health
2	care
2	senate
2	republican
2	states
2	administration
2	republicans
2	public
2	president
2	city
2	year


Finance
11	percent
7	said
6	year
6	company
5	million
4	companies
4	market
3	billion
2	business
2	years
2	according
2	new
2	chief
2	growth
2	investors
2	rate
2	price
2	sales
2	executive
2	prices


Bridge Card Gam

In [54]:
all_years = [avg_weights_2011, avg_weights_2012, avg_weights_2013, avg_weights_2014, avg_weights_2015, avg_weights_2016, avg_weights_2017]
all_years_np = np.array([np.array(x) for x in all_years])

In [81]:
# import seaborn as sns
# import matplotlib.cm as cm
# import matplotlib.style
# NUM_COLORS = 18
# sns.reset_orig()  # get default matplotlib styles back
# # matplotlib.style.use('seaborn')

# clrs = sns.color_palette('Set3', n_colors=NUM_COLORS, desat=.8)  # a list of RGB tuples
# plt.figure(figsize=(15,8))
# for idx, topic in enumerate(all_years_np.T):
#     plt.plot(topic, label=lda_topics[idx], color=clrs[idx])
# #     plt.plot(topic, label=idx)
# plt.xticks(range(7), [2011, 2012, 2013, 2014, 2015, 2016, 2017])
# plt.legend()

In [82]:
for i in range(2011,2018):
    print(i, len(raw_df[raw_df['year'] == i]['article']))

2011 36103
2012 36057
2013 34795
2014 40790
2015 42086
2016 42071
2017 36117


In [2]:
len(raw_df)

NameError: name 'raw_df' is not defined

In [85]:
dicts_2011[2]

{0: 0.139603,
 2: 0.18121177,
 3: 0.025287978,
 4: 0.03069319,
 5: 0.016031202,
 7: 0.285238,
 10: 0.012106614,
 13: 0.030912641,
 14: 0.06993339,
 16: 0.1601656}